In [ ]:
from grid.robot.wheeled.airgen_car import AirGenCar
airgen_car_0 = AirGenCar()

In [ ]:
import h5py, os
from typing import List
from airgen import ImageType, VehicleClient
from airgen.utils.visualize import rr_log_airgen_image
from grid import GRID_USER_SESSION_BLOB_DIR


def capture_data(car: VehicleClient, sensor_types: List[str], capture_types: List[str], hdf5_file_path: str, idx=0):
    with h5py.File(hdf5_file_path, 'a') as hdf5_file:
        frame_group = hdf5_file.create_group(f"frame_{idx}")
        sensor_group = frame_group.create_group("sensor_data")
        image_group = frame_group.create_group("image_data")

        capture_name_map = {
            "rgb": ImageType.Scene, 
            "depth": ImageType.DepthPerspective, 
            "segmentation": ImageType.Segmentation
        }
        
        # Capture IMU data
        if "imu" in sensor_types:
            imu_data = car.getImuData()
            imu_group = sensor_group.create_group("imu")
            imu_group.create_dataset("time_stamp", data=imu_data.time_stamp)
            imu_group.create_dataset("orientation", data=[
                imu_data.orientation.w_val, 
                imu_data.orientation.x_val, 
                imu_data.orientation.y_val, 
                imu_data.orientation.z_val
            ])
            imu_group.create_dataset("angular_velocity", data=[
                imu_data.angular_velocity.x_val, 
                imu_data.angular_velocity.y_val, 
                imu_data.angular_velocity.z_val
            ])
            imu_group.create_dataset("linear_acceleration", data=[
                imu_data.linear_acceleration.x_val, 
                imu_data.linear_acceleration.y_val, 
                imu_data.linear_acceleration.z_val
            ])
        
        # Capture image data
        image_types = [capture_name_map[capture_type] for capture_type in capture_types]
        images = car.getImages("front_center", image_types)
        
        for capture_type, image in zip(capture_types, images):
            rr_log_airgen_image("grid", capture_name_map[capture_type], image[0])
            image_group.create_dataset(capture_type, data=image[0])

    if idx % 10 == 0:
        print(f"Saved {idx} frames")

In [ ]:
import time

# Example usage
hdf5_file_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, 'car_data.h5')
start_time = time.time()

# Apply speed commands to car
airgen_car_0.client.setCarTargetSpeed(1.0)

for i in range(50):
    airgen_car_0.client.simPause(True)
    capture_data(airgen_car_0.client, ['imu'], ['rgb', 'depth', 'segmentation'], hdf5_file_path, idx=i)
    airgen_car_0.client.simPause(False)
    time.sleep(0.1)

airgen_car_0.client.setCarTargetSpeed(0.0)

elapsed_time = time.time() - start_time
print(f"Total time taken: {elapsed_time:.2f} seconds")